In [1]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

from src.array import Array
from src.matrix import Matrix
from src.equation import Equation
from src.failure import Failure

from src.array_new import Array as Array_new
from src.matrix_new import Matrix as Matrix_new
from src.equation_new import Equation as Equation_new
from src.failure_new import Failure as Failure_new

In [2]:
def test1(array, matrix_init, width, seed):
    matrix = Matrix(matrix_init=matrix_init, array=None)
    equation = Equation(array=array, matrix=matrix, save_volts_profile=True)
    failure = Failure(array=array, matrix=matrix, equation=equation, width=width, seed=seed, save_volts_profile=True)

    solve = equation.solve_init
    break_edge = failure.break_edge_init
    solve()
    break_edge()

    solve = equation.solve
    solve_r = equation.solve_r_mmd
    break_edge = failure.break_edge
    for _ in range(array.length - 1):
        solve_r()
        solve()
        break_edge()

    solve_r = equation.solve_r_amd
    while solve_r():
        solve()
        break_edge()

    return matrix, equation, failure

In [3]:
def test2(array_new, matrix_init_new, width, seed):
    matrix_new = Matrix_new(matrix_init=matrix_init_new, array=None)
    equation_new = Equation_new(array=array_new, matrix=matrix_new, save_volts_profile=True)
    failure_new = Failure_new(array=array_new, matrix=matrix_new, equation=equation_new, width=width, seed=seed, save_volts_profile=True)

    #
    solve_new = equation_new.solve_init
    break_edge_new = failure_new.break_edge_init
    solve_new()
    break_edge_new()

    solve_new = equation_new.solve
    solve_r_new = equation_new.solve_r_mmd
    break_edge_new = failure_new.break_edge
    for _ in range(array_new.length - 1):
        solve_r_new()
        solve_new()
        break_edge_new()

    solve_r_new = equation_new.solve_r_amd
    while solve_r_new():
        solve_new()
        break_edge_new()

    return matrix_new, equation_new, failure_new

In [4]:
def check_matrix_init(matrix_init, matrix_init_new, length, width, seed, val_cap, time_step):
    try:
        if not (
            np.all(matrix_init.cond.data == matrix_init_new.cond.data)
            and
            np.all(matrix_init.div_cap.data == matrix_init_new.div_cap.data)
            and
            np.all(matrix_init.div_comb.data == matrix_init_new.div_comb.data)
        ): return (1, length, width, seed, val_cap, time_step)
    except: return (-1, length, width, seed, val_cap, time_step)

In [5]:
def check_data_vs_test(failure, length, width, seed, val_cap, time_step):
    path_seed = Path.cwd() / "data" / str(length) / str(width)/ f"{val_cap}_{time_step}" / str(seed)
    if not path_seed.exists(): return None # filter eg l =  3
    try:
        if not (
            np.all(np.array(failure.volts_ext) == np.load(path_seed / "volts_ext.npy"))
            and
            np.all(np.array(failure.idxs_edge_broken) == np.load(path_seed / "idxs_edge_broken.npy"))
        ): return (2, length, width, seed, val_cap, time_step)
    except: return (-2, length, width, seed, val_cap, time_step) # length no match

In [6]:
def check_matrix(matrix, matrix_new, length, width, seed, val_cap, time_step):
    try:
        if not (
            np.all(matrix.cond.data == matrix_new.cond.data)
            and
            np.all(matrix.div_cap.data == matrix_new.div_cap.data)
            and
            np.all(matrix.div_comb.data == matrix_new.div_comb.data)
        ): return (3, length, width, seed, val_cap, time_step)
    except: return (-3, length, width, seed, val_cap, time_step)

In [7]:
def check_primary_data(failure, failure_new, length, width, seed, val_cap, time_step):
    try:
        if not (
            np.all(np.array(failure.volts_ext) == np.array(failure_new.volts_ext))
            and
            np.all(np.array(failure.idxs_edge_broken) == np.array(failure_new.idxs_edge_broken))
        ): return (4, length, width, seed, val_cap, time_step)
    except: return (-4, length, width, seed, val_cap, time_step)

In [8]:
def check_secondary_data(failure, failure_new, length, width, seed, val_cap, time_step):
    try:
        if not (
            np.all(np.array(failure.volts_cap_profile) == np.array(failure_new.volts_cap_profile))
            and
            np.all(np.array(failure.volts_cond_profile) == np.array(failure_new.volts_cond_profile)) ############## failuire_new * -1 ? 
            and
            np.all(np.array(failure.volts_edge_profile) == np.array(failure_new.volts_edge_profile))
        ): return (5, length, width, seed, val_cap, time_step)
    except: return (-5, length, width, seed, val_cap, time_step)

---

In [9]:
checks = []

---

In [ ]:
val_cap, time_step = 1.0, 0.01
val_cap, time_step = float(val_cap), float(time_step)

for length in [3, 7, 9, 10, 20, 30, 40]:
    array = Array(length=length, mode_analysis=True)
    matrix_init = Matrix(matrix_init=None, array=array, val_cap=val_cap, time_step=time_step)
    array_new = Array_new(length=length, mode_analysis=True)
    matrix_init_new = Matrix_new(matrix_init=None, array=array_new, val_cap=val_cap, time_step=time_step)

    # CHECK 1
    check_1 = check_matrix_init(matrix_init, matrix_init_new, length, np.nan, np.nan, val_cap, time_step)
    if check_1 is not None: checks.append(check_1)

    if length < 10: continue
    for width in np.arange(0.25, 2.25, 0.25).tolist() + [0.85, 1.15]:
        print((length, width), len(checks))
        for seed in list(range(100)) + [333, 444, 555, 666, 777, 888, 999, 1000]:

            matrix, equation, failure = test1(array, matrix_init, width, seed)

            # CHECK 2
            check_2 = check_data_vs_test(failure, length, width, seed, val_cap, time_step)
            if check_2 is not None: checks.append(check_2)

            matrix_new, equation_new, failure_new = test2(array_new, matrix_init_new, width, seed)

            # CHECK 3
            check_3 = check_matrix(matrix, matrix_new, length, width, seed, val_cap, time_step)
            if check_3 is not None: checks.append(check_3)

            # CHECK 4
            check_4 = check_primary_data(failure, failure_new, length, width, seed, val_cap, time_step)
            if check_4 is not None: checks.append(check_4)

            # CHECK 5
            check_5 = check_secondary_data(failure, failure_new, length, width, seed, val_cap, time_step)
            if check_5 is not None: checks.append(check_5)

(10, 0.25) 0
(10, 0.5) 0
(10, 0.75) 0
(10, 1.0) 0
(10, 1.25) 0
(10, 1.5) 0


KeyboardInterrupt: 

---

In [ ]:
length = 3
width = 1.25
seed = 23
val_cap = 1.0
time_step = 0.01

array = Array(length=length, mode_analysis=True)
matrix_init = Matrix(matrix_init=None, array=array, val_cap=val_cap, time_step=time_step)
array_new = Array_new(length=length, mode_analysis=True)
matrix_init_new = Matrix_new(matrix_init=None, array=array_new, val_cap=val_cap, time_step=time_step)

matrix, equation, failure = test1(array, matrix_init, width, seed)
matrix_new, equation_new, failure_new = test2(array_new, matrix_init_new, width, seed)